# Creating Datasets

In this example we will show how `openff-qcsubmit` can be used to curate [QCFractal](http://docs.qcarchive.molssi.org/projects/qcfractal/en/latest/)
compatible datasets of molecules that can be submitted to public or local fractals instances (e.g. the [QCArchive](https://qcarchive.molssi.org/)).

In particular, we aim to create reproducible workflows that can process large lists of molecules, applying filtering and
some other useful operations such as state enumeration and fragmentation. The entire workflow is built out of modular
components which can each have programmable settings that can be controlled through the API or via workflow yaml/json
files.

We will demonstrate how to set up one such workflow via the API and how it can be exported to a settings file that can
then be used to reconstruct the entire workflow by another user.

For the sake of clarity all verbose will be disabled in this tutorial:

In [1]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

## Creating a dataset factory

The `openff-qcsubmit` package provides a number of dataset 'factories'. A factory is a reusable object that encodes all
of the core settings that will be used to curate / compute a dataset from an input list of molecule.

Here we will begin by creating a 'basic' data set factory:

In [2]:
from qcportal.models.common_models import DriverEnum

from openff.qcsubmit.factories import BasicDatasetFactory
from openff.qcsubmit.common_structures import QCSpec

factory = BasicDatasetFactory(
    driver=DriverEnum.energy,
    qc_specifications={
        "default": QCSpec(),
        "ani1ccx": QCSpec(
            program="torchani",
            method="ani1ccx",
            basis=None,
            spec_name="ani1ccx",
            spec_description="ANI1ccx standard specification"
        )
    }
)

# lets look at the class and the settings 
factory

BasicDatasetFactory(qc_specifications={'default': QCSpec(method='B3LYP-D3BJ', basis='DZVP', program='psi4', spec_name='default', spec_description='Standard OpenFF optimization quantum chemistry specification.', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>, implicit_solvent=None), 'ani1ccx': QCSpec(method='ani1ccx', basis=None, program='torchani', spec_name='ani1ccx', spec_description='ANI1ccx standard specification', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>, implicit_solvent=None)}, maxiter=200, driver=<DriverEnum.energy: 'energy'>, scf_properties=[<SCFProperties.Dipole: 'dipole'>, <SCFProperties.Quadrupole: 'quadrupole'>, <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>, <SCFProperties.MayerIndices: 'mayer_indices'>], priority='normal', dataset_tags=['openff'], compute_tag='openff', type='BasicDatasetFactory', workflow=[])

this factory will be responsible for creating a 'basic' dataset that will contain a collection of single point
calculations provided through the energy/gradient/hessian drivers.

Here we have specified that datasets created using this factory should be computed using two different 'quantum chemical'
(QC) specfications:

* `default`: the OpenFF default specification which employs `B3LYP-D3BJ+DZVP` using `psi4`.
* `ani1ccx`: ANI1ccx provided by the `torchani` package.

Note that the basic settings should be suitable in most cases as they are those recommended by the OpenFF Consortium and
are currently used in the fitting of the most recent force fields.

Dataset factories are also available for optimization and torsion drive data sets.

Now lets look at the workflow components that will be used to curate our initial set of molecules.

In [3]:
factory.workflow

[]

The workflow is a dictionary that contains all of the components that will be executed in order, and by default will be
empty.

`openff-qcsubmit` provides a suite of common curation components, such as to filter out molecules that contain
unsupported elements, or to generate a set of conformers for each molecule.

Here we will set up a workflow that will filter out some unwanted elements (i.e. those not supported by ANI1), then
filter by molecular weight, and finally generate conformers for each of the molecules passing through the factory.

First we set up the element filter:

In [4]:
from openff.qcsubmit import workflow_components

el_filter = workflow_components.ElementFilter(
    allowed_elements=[1, 6, 7, 8]
)
# lets view the options available for this filter
el_filter

factory.add_workflow_components(el_filter)

This filter has the ability to filter elements by atomic name or number, we just have to supply a list of 
symbols or numbers to the filter. Here we only keep molecules with elements of H,C,N and O as we would like
to use AN1 as our QC method.

Now we will set up the weight filter and conformer generation components and add them to the workflow.

In [5]:
weight_filter = workflow_components.MolecularWeightFilter()
factory.add_workflow_components(weight_filter)

conf_gen = workflow_components.StandardConformerGenerator(
    max_conformers=1,
    toolkit="rdkit"
)
factory.add_workflow_components(conf_gen)

Let's look at the workflow and make sure all the components were correctly added:

In [6]:
factory.workflow

[ElementFilter(type='ElementFilter', allowed_elements=[1, 6, 7, 8]),
 MolecularWeightFilter(type='MolecularWeightFilter', minimum_weight=130, maximum_weight=781),
 StandardConformerGenerator(type='StandardConformerGenerator', toolkit='rdkit', rms_cutoff=None, max_conformers=1, clear_existing=True)]

and save the settings and workflow so they can be used again later:

In [7]:
factory.export_settings("example-factory.json")
factory.export_settings("example-factory.yaml")

While workflows can be exported to several different formats, the most commonly used are JSON and YAML as shown in this
example. Let's look at the JSON output:

In [8]:
! head -n 20 work1.json

head: work1.json: No such file or directory


These settings can be re-imported easily using the API:

In [9]:
imported_factory = BasicDatasetFactory.from_file("example-factory.json")

## Creating the dataset

We can run the workflow on an example set of molecules:

In [10]:
from openff.toolkit.topology import Molecule

mols = [
    Molecule.from_smiles(smiles)
    for smiles in [
        "[H]/N=C(/N)\\Nc1[nH]nnn1",
        "c1cc[nH+]cc1",
        "C[N+](C)(C)[O-]",
        "CONC(=O)N",
        "c1ccc2c(c1)cc[nH]2",
        "c1ccc(cc1)/N=C\\NO",
        "C=CO",
        "c1cocc1[O-]",
        "CC(=O)NO",
        "C[N+](=C)C",
        "C(=O)C=O",
        "C=C",
        "CC1=NC(=NC1=[N+]=[N-])Cl",
        "c1cc[n+](cc1)[O-]",
        "CN(C)O",
        "N(=O)(=O)O",
        "CC=O",
        "c1cc(oc1)c2ccco2",
        "CC",
        "C1C=CC(=O)C=C1",
    ]
]

This is as simple as calling the factories `create_dataset` method and providing the set of molecules as input:

In [11]:
dataset = factory.create_dataset(
    molecules=mols,
    dataset_name="example-dataset",
    description="An example dataset.",
    tagline="An example dataset."
)
dataset

Preparation                   : 100%|█████████████| 2/2 [00:00<00:00, 68.54it/s]


BasicDataset(qc_specifications={'default': QCSpec(method='B3LYP-D3BJ', basis='DZVP', program='psi4', spec_name='default', spec_description='Standard OpenFF optimization quantum chemistry specification.', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>, implicit_solvent=None), 'ani1ccx': QCSpec(method='ani1ccx', basis=None, program='torchani', spec_name='ani1ccx', spec_description='ANI1ccx standard specification', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>, implicit_solvent=None)}, maxiter=200, driver=<DriverEnum.energy: 'energy'>, scf_properties=[<SCFProperties.Dipole: 'dipole'>, <SCFProperties.Quadrupole: 'quadrupole'>, <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>, <SCFProperties.MayerIndices: 'mayer_indices'>], priority='normal', dataset_tags=['openff'], compute_tag='openff', dataset_name='example-dataset', dataset_tagline='An example dataset.', dataset_type='DataSet', description='An example dataset.', metadata=Metadata(submitter='boothros'

We can easily see how many molecules the dataset contains after filtering:

In [12]:
dataset.n_molecules

2

and how many QC 'records' will be computed for this dataset:

In [13]:
dataset.n_records

2

We can iterate over the molecules in the dataset:

In [14]:
for molecule in dataset.molecules:
    print(molecule.to_smiles(explicit_hydrogens=False))

c1ccc(cc1)/N=C\NO
C1=COC(=C1)C2=CC=CO2


as well as those that were filtered out during its construction:

In [15]:
for molecule in dataset.filtered:
    print(molecule.to_smiles(explicit_hydrogens=False))

CC1=NC(=NC1=[N+]=[N-])Cl
[H]/N=C(/N)\NC1=NN=NN1
c1cc[nH+]cc1
C[N+](C)(C)[O-]
CONC(=O)N
c1ccc2c(c1)C=CN2
C=CO
C1=COC=C1[O-]
CC(=O)NO
C[N+](=C)C
C(=O)C=O
C=C
c1cc[n+](cc1)[O-]
CN(C)O
N(=O)(=O)O
CC=O
CC
C1C=CC(=O)C=C1


The fully created dataset is readily exportable to JSON:

In [16]:
dataset.export_dataset("example-dataset.json")

and the molecules it contains can be exported to various formats:

In [17]:
dataset.molecules_to_file("example-dataset.smi", "smi")
dataset.molecules_to_file("example-dataset.inchi", "inchi")
dataset.molecules_to_file("example-dataset.inchikey", "inchikey")

The molecules contained within a dataset can also be easily visualized by exporting the dataset to a PDF:

In [18]:
dataset.visualize("example-dataset.pdf")